# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [ ]:
# Load the libraries as required.
import dask.dataframe as dd
import pandas as pd
import numpy as np

In [ ]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [ ]:
X=fires_dt.drop(columns = 'area')

In [ ]:
Y=fires_dt['area']

In [ ]:
X.head()


,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


In [ ]:
num_cols = [
    'coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 
    'isi', 'temp', 'rh', 'wind', 'rain'
]

cat_cols = ['month', 'day'] 


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
pipe_num_simple =Pipeline([('imputer',SimpleImputer(strategy='median')),
                     ('standardizer',StandardScaler())
                     ])

In [ ]:
cat_pipe=Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                   ('encoder', OneHotEncoder(handle_unknown='ignore'))

])

In [ ]:
preproc1=ColumnTransformer([('num', pipe_num_simple, num_cols),
                            ('cat', cat_pipe, cat_cols)

], remainder='passthrough')

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [ ]:
num_cols_transform_selected = [
    'ffmc', 'dmc', 'dc', 'isi', 'temp'
]
num_cols_transform_rest = [
    'coord_x', 'coord_y', 'rh', 'wind', 'rain'
]


In [ ]:
num_pipe_2=Pipeline([('imputer',SimpleImputer(strategy='median')),
                     ('transform',PowerTransformer(method='yeo-johnson')),
                     ('standardizer', StandardScaler())
                     ])

In [ ]:
preproc2=ColumnTransformer([
    ('numeric_std', pipe_num_simple, num_cols_transform_rest),
    ('numeric_yj', num_pipe_2, num_cols_transform_selected),
    ('cat', cat_pipe, cat_cols)
],remainder='passthrough')

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate


In [ ]:
# Pipeline A = preproc1 + baseline
pipe_lr_proc1=Pipeline([
   ('preprocessing', preproc1),
   ('regressor', LinearRegression())
])

In [ ]:
# Pipeline B = preproc2 + baseline
pipe_lr_proc2=Pipeline([
   ('preprocessing', preproc2),
   ('regressor', LinearRegression())
])


In [ ]:
# Pipeline C = preproc1 + advanced model
pipe_rf_proc1=Pipeline([
   ('preprocessing', preproc1),
   ('regressor', RandomForestRegressor())
])

In [ ]:
# Pipeline D = preproc2 + advanced model
pipe_rf_proc2=Pipeline([
   ('preprocessing', preproc2),
   ('regressor', RandomForestRegressor())
]) 

In [ ]:

scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
res_lr_proc1_dict = cross_validate(pipe_lr_proc1, X_train, Y_train, cv = 5, scoring = scoring)
res_lr_proc1 = pd.DataFrame(res_lr_proc1_dict).assign(experiment = 1)
res_lr_proc1
res_lr_proc1.mean()

fit_time                           0.010401
score_time                         0.006926
test_r2                           -0.315601
test_neg_mean_squared_error    -2230.724512
test_neg_mean_absolute_error     -18.486499
experiment                         1.000000
dtype: float64

In [ ]:
scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
res_rf_proc1_dict = cross_validate(pipe_rf_proc1, X_train, Y_train, cv = 5, scoring = scoring)
res_rf_proc1 = pd.DataFrame(res_rf_proc1_dict).assign(experiment = 1)
res_rf_proc1.mean()

fit_time                           0.244332
score_time                         0.016000
test_r2                           -1.700644
test_neg_mean_squared_error    -2795.537600
test_neg_mean_absolute_error     -20.143412
experiment                         1.000000
dtype: float64

In [ ]:
scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
res_lr_proc2_dict = cross_validate(pipe_lr_proc2, X_train, Y_train, cv = 5, scoring = scoring)
res_lr_proc2 = pd.DataFrame(res_lr_proc2_dict).assign(experiment = 1)
res_lr_proc2.mean()

fit_time                           0.020124
score_time                         0.010200
test_r2                           -0.315275
test_neg_mean_squared_error    -2222.291105
test_neg_mean_absolute_error     -18.475709
experiment                         1.000000
dtype: float64

In [ ]:
scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
res_rf_proc2_dict = cross_validate(pipe_rf_proc2, X_train, Y_train, cv = 5, scoring = scoring)
res_rf_proc2 = pd.DataFrame(res_rf_proc2_dict).assign(experiment = 1)
res_rf_proc2.mean()

fit_time                           0.270371
score_time                         0.017765
test_r2                           -1.758112
test_neg_mean_squared_error    -2877.776671
test_neg_mean_absolute_error     -20.152915
experiment                         1.000000
dtype: float64

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.